<p style="text-align:center">
PSY 381D <b>Brain Connectivity</b>, Spring 2019


<img style="width: 700px; padding: 0px;" src="https://github.com/sathayas/JupyterConnectivitySpring2019/blob/master/Images/Banner.png?raw=true" alt="title pics"/>

</p>

<p style="text-align:center; font-size:40px; margin-bottom: 30px;"><b> Small-world networks</b></p>

<p style="text-align:center; font-size:18px; margin-bottom: 32px;"><b>February 4, 2019</b></p>

<hr style="height:5px;border:none" />

# Network data
<hr style="height:1px;border:none" />

We will be examining these network data for today's exercises. They are available in the **`DataSmallWorld`** directory.
* C. Elegans neural network (`CElegans.adjlist`)
* Power grid (`power.gml`)
* Brain network (resting-state fMRI)
   * ROI network (`Oxford_sub16112_aal90_d5.adjlist`)
   * Voxel network (`Oxford_sub16112_voxel_d20.adjlist`)

`<NetworkSize.py>`

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

##### loading the network data
# C Elegans neural network
G_CEleg = nx.read_adjlist('DataSmallWorld/CElegans.adjlist')
# Power grid
G_Power = nx.read_gml('DataSmallWorld/power.gml', label='id')
# Brain (ROI)
G_ROI = nx.read_adjlist('DataSmallWorld/Oxford_sub16112_aal90_d5.adjlist')
# Brain (Voxel)
G_Voxel = nx.read_adjlist('DataSmallWorld/Oxford_sub16112_voxel_d20.adjlist')


   
### Exercise
**Number of nodes & edges**: Find the missing numbers in the table below, by determining the number of nodes or edges, or the average degree (2 $\times$ number of edges divided by the number of nodes). ***Post the numbers on Canvas.***

<table>
<tr>
<th style="text-align:left">Network</th>
<th style="text-align:center">Number of nodes</th>
<th style="text-align:center">Number of edges</th>
<th style="text-align:center">Average degree</th>
</tr>
<tr>
<td style="text-align:left">C. Elegans</td>
<td style="text-align:center">297</td>
<td style="text-align:center">2148</td>
<td style="text-align:center"><b>(a)</b></td>
</tr>
<tr>
<td style="text-align:left">Power grid</td>
<td style="text-align:center"><b>(b)</b></td>
<td style="text-align:center">6594</td>
<td style="text-align:center">2.7</td>
</tr>
<tr>
<td style="text-align:left">Brain (ROI)</td>
<td style="text-align:center">90</td>
<td style="text-align:center"><b>(c)</b></td>
<td style="text-align:center"><b>(d)</b></td>
</tr>
<tr>
<td style="text-align:left">Brain (Voxel)</td>
<td style="text-align:center"><b>(e)</b></td>
<td style="text-align:center">295219</td>
<td style="text-align:center">30.7</td>
</tr>
</table>

# Clustering coefficient
<hr style="height:1px;border:none" />

The **clustering coefficient** of a network can be calculated by the **`average_clustering`** function.

`<ClusteringCoefficient.py>`

In [2]:
##### Clustering coefficients
print('Clustering coefficients')
print('C. Elegans: %4.2f' % nx.average_clustering(G_CEleg))
print('Power grid: %5.3f' % nx.average_clustering(G_Power))
print('Brain (ROI): %4.2f' % nx.average_clustering(G_ROI))
print('Brain (Voxel): %4.2f' % nx.average_clustering(G_Voxel))

Clustering coefficients
C. Elegans: 0.29
Power grid: 0.080
Brain (ROI): 0.51
Brain (Voxel): 0.27


# Path length
<hr style="height:1px;border:none" />

The average shortest path length (or simply referred as **path length**) of a network can be calculated by the **`average_shortest_path_length`** function. *Calculation of path length is computationally very demanding, especially for a large network or a densely connected network*. 

`<PathLength.py>`

In [3]:
##### Path Length
print('Average shortest path lengths')
print('C. Elegans: %4.2f' % nx.average_shortest_path_length(G_CEleg))
print('Power grid: %4.2f' % nx.average_shortest_path_length(G_Power))
print('Brain (ROI): %4.2f' % nx.average_shortest_path_length(G_ROI))
print('Brain (Voxel): %4.2f' % nx.average_shortest_path_length(G_Voxel))

Average shortest path lengths
C. Elegans: 2.46
Power grid: 18.99


NetworkXError: Graph is not connected.

You see an error. This is because the brain network (ROI) is not **connected**. (**Connected:** all nodes are connected to a single network without any disconnected component). Path length cannot be calculated if some nodes cannot be reached from some other nodes due to disconnection. 

We can check the connected component sizes using **`connected_components`** function.

In [4]:
#### Checking the connected components
ccSize_ROI = [len(c) for c in sorted(nx.connected_components(G_ROI),
                                     key=len,
                                     reverse=True)]
print('Brain (ROI), connected component sizes: ', ccSize_ROI)

Brain (ROI), connected component sizes:  [89, 1]


In [ ]:
ccSize_Voxel = [len(c) for c in sorted(nx.connected_components(G_Voxel),
                                       key=len,
                                       reverse=True)]
print('Brain (Voxel), connected component sizes: ', ccSize_Voxel)

We can calculate the path length on the largest connected component (a.k.a., **giant component**) only.

In [6]:
##### Path length, giant component only
GC_nodes_ROI = max(nx.connected_components(G_ROI), key=len)  # nodes in giant component
GC_ROI = G_ROI.subgraph(GC_nodes_ROI)  # nodes & edges in giant component

The network **`GC_ROI`** only contains nodes and edges in the giant component.

In [7]:
len(GC_ROI.nodes())

89

In [8]:
len(GC_ROI.edges())

279

In [9]:
print('Path length, brain (ROI): %4.2f'  % nx.average_shortest_path_length(GC_ROI))

Path length, brain (ROI): 3.76


### Exercise
**Path length, brain (voxel) network**: Calculate the number of nodes and edges, as well as the path length of the giant component of the brain (voxel) network. ***Post the numbers on Canvas.***

# Standardized clustering coefficient and path length
<hr style="height:1px;border:none" />

You may notice by now that the magnitude of clustering coefficients C and path lengths L varies tremendously for different networks. To determine whether a network exhibits *small-world* characteristics, it is necessary to compare C and L against that obtained from an equivalent random network (with the same number of nodes, edges, and degree sequence).

## Random network with a given degree sequence
It is important to compare C and L from a network of interest to an equivalent random network with matching nodes, edges, and degree sequence. This is to rule out any difference between the two networks due to the difference in the availability of nodes / edges and how they are connected. In other words, we want to compare an apple with another apple (an apple lacking the organization principle).

We can generate an equivalent random network by degree-preserving re-wiring (Maslov & Sneppen) 
* Figure for rewiring
* Degree sequence, before & after
* Connected network -- hard to guarantee 
   * Giant component only
* Examples
* Small-worldness by Humphries


* Clustering & path length
   * Real networks
   * Random networks
      * ER network
      * WS network
* Small-worldness
   * Random model
      * Theoretical (Watts & Strogatz)
         * Degree distribution
      * Re-wired model (Maslov & Sneppen)
* Brain networks (fMRI)
    * ROI network
         * with and without disconnected
    * Voxel network
